# Import

In [519]:
%matplotlib qt
%gui qt

import re
import os
import sys

import numpy as np
from loguru import logger

import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches


import pyqtgraph as pg
import pyqtgraph.opengl as gl

from collections import defaultdict
from datetime import datetime

import torch
import gpytorch
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import scipy.interpolate as interp

In [520]:
def gatherCSV(rootPath, outsuffix = 'Tracking'):
    '''==================================================
        Collect all EIS.csv files in the rootPath
        Parameter: 
            rootPath: current search path
            outsuffix: Saving path of EIS.csv files
        Returen:
            EISDict: a 2D-dict of EIS data
            Storage Frame: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        ==================================================
    '''
    _filename       = None
    _filepath       = None
    _trackpath      = None
    _csvpath        = None
    _sessionIndex   = None
    _channelIndex   = None
    _processed      = None

    EISDict = defaultdict(dict)

    ## Iterate session
    session_pattern = re.compile(r"(.+?)_(\d{8})_01")
    bank_pattern    = re.compile(r"([1-4])")
    file_pattern    = re.compile(r"EIS_ch(\d{3})\.csv")

    ## RootDir
    for i in os.listdir(rootPath):
        match_session = session_pattern.match(i)
        ## SessionDir
        if match_session:
            logger.info(f"Session Begin: {i}")
            _sessionIndex = match_session[2]
            for j in os.listdir(f"{rootPath}/{i}"):
                match_bank = bank_pattern.match(j)
                ## BankDir
                if match_bank:
                    logger.info(f"Bank Begin: {j}")
                    _trackpath = f"{rootPath}/{i}/{j}/{outsuffix}"
                    if not os.path.exists(_trackpath):
                        continue

                    for k in os.listdir(f"{rootPath}/{i}/{j}/{outsuffix}"):
                        match_file = file_pattern.match(k)
                        ## File
                        if match_file:
                            _filename = k
                            _filepath = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            _channelIndex = (int(match_bank[1])-1)*32+int(match_file[1])
                            
                            EISDict[_sessionIndex][_channelIndex] = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            
    return EISDict

In [521]:
# Data Readout
def readChannel(chID, fileDict):
    '''==================================================
        Read EIS.csv file by Channel
        Parameter: 
            chID: channel index
            fileDict: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        Returen:
            freq: frequency
            Zreal: real part of impedance
            Zimag: imaginary part of impedance
        ==================================================
    '''
    chData = []
    for ssID in fileDict.keys():
        _data   = np.loadtxt(fileDict[ssID][chID], delimiter=',')
        _freq   = _data[:,0]
        _Zreal  = _data[:,1] * np.cos(np.deg2rad(_data[:,2])) 
        _Zimag  = _data[:,1] * np.sin(np.deg2rad(_data[:,2])) 
        chData.append(np.stack((_freq, _Zreal, _Zimag),axis=0))

    return np.stack(chData, axis=0)

In [522]:


def EIS_recal_ver02(data, _phz_0 = None):
    f_poi = data[0,:]
    # Z_poi = data[1,:] * np.exp(1j*np.deg2rad(data[2,:]))
    Z_poi = data[1,:] + 1j*data[2,:]
    Y_poi = 1/Z_poi

    Rg0 = 1.611e13
    Cp0 = 1.4e-9
    
    _Rg0_rescale = 1/Rg0*np.power(f_poi,1.583)
    _Cp0_rescale = Cp0*np.power(f_poi,0.911)
    Y_org = Y_poi - _Rg0_rescale + 1j*_Cp0_rescale
    # Y_org = Y_poi - _Rg0_rescale 
    # Y_org = Y_poi + 1j*_Cp0_rescale
    # Y_org = Y_poi
    Z_org = 1/Y_org

    # Phz Calibration
    if _phz_0 is None:
        _phz_0 = np.loadtxt("./phz_Calib.txt")
    
    Z_ampC = np.abs(Z_org)
    # Z_phzC = np.angle(Z_org) - _phz_0
    Z_phzC = np.angle(Z_org) - _phz_0

    Z_rec = Z_ampC * np.exp(1j*Z_phzC)

    # C = 5e-10
    Rs0 = 100
    Z_rec = Z_rec - Rs0



    Cp0 = 5e-10
    _Cp0_rescale = Cp0 * f_poi
    Z_rec = 1/(1/Z_rec - 1j * _Cp0_rescale)

    

    # Ls0 = 1.7e-4
    Ls0 = 5e-4
    _Ls0_rescale = Ls0 * f_poi
    Z_rec = Z_rec - 1j * _Ls0_rescale

    # C = 5e-10
    Rs0 = 566
    Z_rec = Z_rec - Rs0
    
    return np.stack([f_poi, np.real(Z_rec), np.imag(Z_rec)], axis=1).T
    

## Data Read-in

In [523]:
# rootPath = "D:/Baihm/EISNN/Dataset/01037160_归档"
# ch_id = 20  # Normal to Short, Same to GPR  
# ch_id = 89  # Same to GPR  
# ch_id = 7  # Normal Example

# rootPath = "D:/Baihm/EISNN/Dataset/05087163_归档"
# ch_id = 7   # one outlier
# ch_id = 50  # No outlier but in two Phases
# ch_id = 55  # One outlier &wired end point
# ch_id = 114 # Open Circuit with on outpler

# rootPath = "D:/Baihm/EISNN/Archive/02067447_归档"
# ch_id = 68  # Short all the time

# rootPath = "D:/Baihm/EISNN/Archive/01067095_归档"
# ch_id = 19    # First Sample is outlier

rootPath = "D:/Baihm/EISNN/Archive/09290511_归档"
ch_id = 13    # Up & Down, 2 outliers
# ch_id = 21    # Normal + 2 outlier
# ch_id = 41    # Normal + 2 outlier - *(Hard To Tell)
# ch_id = 79    # 3-class, What a mess

# rootPath = "D:/Baihm/EISNN/Archive/11057712_归档"
# ch_id = 106    # Very Good Electrode with 1 hidden outlier, and one phase shift

# rootPath = "D:\Baihm\EISNN\Archive/10057084_归档"
# ch_id = 16    # Totaly Mess
# ch_id = 18    # Totaly Mess

# rootPath = "D:\Baihm\EISNN\Archive/11067223_归档"
# ch_id = 124     # Perfect with one outlier

# rootPath = "D:\Baihm\EISNN\Archive/06017758_归档"
# ch_id = 96     # Perfect of Perfect

# rootPath = "D:\Baihm\EISNN\Archive/15361101_归档"
# ch_id = 0     # Only One Sample - Run With Error


# rootPath = "D:\Baihm\EISNN\Archive/11207147_归档"
# ch_id = 0     # Only Three Sample - Run With Error

# freq_list = np.linspace(0,np.shape(chData)[2]-1,101,dtype=int)
freq_list = np.linspace(0,5000-1,101,dtype=int, endpoint=True)
EISDict = gatherCSV(rootPath)
chData = readChannel(ch_id, EISDict)

if False:
    phz_calibration = np.loadtxt("./phz_Calib.txt")
    for i in range(np.shape(chData)[0]):
        ch_eis = EIS_recal_ver02(chData[i,:,:], phz_calibration)
        chData[i,:,:] = ch_eis
chData = chData[:,:,freq_list]

# chData = chData[:,:,91:100]


np.shape(chData)
        

2025-04-04 18:21:54.999 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241022_01
2025-04-04 18:21:55.000 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-04-04 18:21:55.001 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-04-04 18:21:55.003 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-04-04 18:21:55.004 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-04-04 18:21:55.005 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241024_01
2025-04-04 18:21:55.006 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-04-04 18:21:55.007 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-04-04 18:21:55.008 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-04-04 18:21:55.008 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4


2025-04-04 18:21:55.009 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241028_01
2025-04-04 18:21:55.009 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-04-04 18:21:55.010 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-04-04 18:21:55.010 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-04-04 18:21:55.011 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-04-04 18:21:55.012 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241029_01
2025-04-04 18:21:55.013 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-04-04 18:21:55.014 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-04-04 18:21:55.014 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-04-04 18:21:55.015 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-04-04 18:21:55.016 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241030_01
2025-04-04 18:21:55.017 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-04-04 18:21:55.017 | INFO    

(13, 3, 101)

## Data Cleaning

In [524]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from  Outlier import OutlierDetection

CLEAN_FLAG = True
if CLEAN_FLAG:
    eis_seq, eis_cluster, eis_anomaly = OutlierDetection.OutlierDetection(chData)
else: 
    eis_seq = np.arange(np.shape(chData)[0])

## Data Plot

In [525]:
if False:
    fig, axis = plt.subplots(1,4,figsize=(15,6))
    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(np.shape(chData)[0]):
    # for i in [1]:
        ch_eis = chData[i,:,:]
        # ch_eis = EIS_recal(chData[i,:,:].T).T
        # ch_eis = EIS_recal_ver02(chData[i,:,:], phz_calibration)
        _color = cmap(i/np.shape(chData)[0])
        axis[0].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"Session {i}")
        axis[1].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"Session {i}")
        axis[2].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2, label=f"Session {i}")
        # axis[4].loglog(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2, label=f"Session {i}")
    
        # _poi_Z = np.log(np.abs(ch_eis[1,:]+1j*ch_eis[2,:]))
        # _poi_P = np.angle(ch_eis[1,:]+1j*ch_eis[2,:])
        # _poi_eis = _poi_Z * np.exp(1j*_poi_P)
        # axis[3].plot(np.real(_poi_eis), -np.imag(_poi_eis), color = _color, linewidth=2, label=f"Session {i}")
        _poi_Z = np.log(ch_eis[1,:]+1j*ch_eis[2,:])
        axis[3].plot(np.real(_poi_Z), -np.imag(_poi_Z), color = _color, linewidth=2, label=f"Session {i}")
        

# axis[0].legend(frameon=False, loc='upper left')

In [526]:
if True:
    fig= plt.figure(figsize=(15,8), constrained_layout=False)
    axis = [0] * 8
    axis[0] = fig.add_subplot(2,4,1, projection='3d')   
    axis[1] = fig.add_subplot(2,4,2)            
    axis[2] = fig.add_subplot(2,4,3)         
    axis[3] = fig.add_subplot(2,4,4)      
    axis[4] = fig.add_subplot(2,4,5, projection='3d')      
    axis[5] = fig.add_subplot(2,4,6)         
    axis[6] = fig.add_subplot(2,4,7)         
    axis[7] = fig.add_subplot(2,4,8)    

    init_elev = 21  # 仰角
    init_azim = 55  # 方位角
    axis[0].view_init(elev=init_elev, azim=init_azim)
    axis[4].view_init(elev=init_elev, azim=init_azim)


    num_samples = np.shape(chData)[0]

    _x = np.arange(num_samples)[eis_seq]
    _y = np.log10(chData[0,0,:]).flatten()
    X, Y = np.meshgrid(_x, _y, indexing='ij')
    axis[0].plot_surface(X, Y, np.log10(np.abs(chData[eis_seq,1,:]+1j*chData[eis_seq,2,:])), cmap='viridis_r', alpha=0.8)
    axis[4].plot_surface(X, Y, np.rad2deg(np.angle(chData[eis_seq,1,:]+1j*chData[eis_seq,2,:])), cmap='viridis_r', alpha=0.8)



    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(len(eis_seq)):
        _x = eis_seq[i]
        ch_eis = chData[_x,:,:]
        _color = cmap(_x/num_samples)
        axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"S{i:02d}")
        axis[5].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"S{i:02d}")


    cmap = plt.colormaps.get_cmap('Set1')
    for i in range(len(eis_seq)):
        _x = eis_seq[i]
        ch_eis = chData[_x,:,:]
        _color = cmap(eis_cluster[i])
        axis[2].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"{chr(ord('A')+eis_cluster[i])}")
        axis[6].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"{chr(ord('A')+eis_cluster[i])}")

    _legend_handle = []
    for i in range(len(np.unique(eis_cluster))):
        _legend_handle.append(mpatches.Patch(color = cmap(i), label = f"{chr(ord('A')+i)}:{len(eis_cluster[eis_cluster==i])}"))
    axis[2].legend(handles=_legend_handle)

    axis[2].sharex(axis[1])
    axis[6].sharex(axis[5])


    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(len(eis_anomaly)):
        _x = eis_anomaly[i]
        ch_eis = chData[_x,:,:]
        _color = cmap(_x/num_samples)
        axis[3].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"S{_x:02d}")
        axis[7].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"S{_x:02d}")
    axis[3].legend()
    axis[3].sharex(axis[1])
    axis[7].sharex(axis[5])



# GPR

## Complex GP Model

In [539]:
# Single Point  Gaussian Process Regression
# 这个脚本中，我们把不同频率视为相互独立的变量进行考察
# 由于阻抗仍是一个复数，所以这里使用实部虚部两个task进行GP

class ComplexGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=2
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            # gpytorch.kernels.RBFKernel(),
            # gpytorch.kernels.RQKernel(),
            gpytorch.kernels.MaternKernel(nu=2.5),
            num_tasks=2, rank=2
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)


def ComplexGPTrain(x_train, y_train, x_eval, device, training_iter = 50, lr = 0.05):
   
   # Initialize likelihood and model
    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=2, rank=2).to(device)
    model = ComplexGPModel(x_train, y_train, likelihood).to(device)

    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # logger.info(f"Training for {training_iter} iterations...")
    loss_inst       = []
    length_inst     = []
    noise_inst      = []
    # alpha_inst      = []
    for i in range(training_iter):
        optimizer.zero_grad()
        output = model(x_train)
        loss = -mll(output, y_train)
        loss.backward()
        optimizer.step()
        
        poi_noise   = model.likelihood.noise.item()
        poi_length  = model.covar_module.data_covar_module.lengthscale.item()
        # poi_alpha   = 0
        # poi_alpha   = model.covar_module.data_covar_module.alpha.item()
        loss_inst.append(loss.item())
        length_inst.append(poi_length)
        # alpha_inst.append(poi_alpha)
        noise_inst.append(poi_noise)
        if not i % 10:
            logger.info(f"Iter {i+1}/{training_iter}\tLoss: {loss.item()}\tLength: {poi_length}\tNoise: {poi_noise}")
    
    # logger.info("Model Training Finished.")

    # Make predictions
    model.eval()
    likelihood.eval()
    with torch.no_grad(), gpytorch.settings.cholesky_jitter(1e-4):
        pred = likelihood(model(x_eval))
    # logger.info("Model Evaluation Finished.")

    return [pred, np.array(loss_inst), np.array(length_inst), np.array(noise_inst)]



## Main

In [540]:
def GPDataLoader(chData, eis_seq, eis_cluster, run_list, SPEED_RATE = 1, LOG_FLAG = True, NORM_X_FLAG = True, NORM_Y_FLAG = True):
    # Speed Rate = 10 means 1 day = 10 points
    x_day = [datetime.strptime(date, '%Y%m%d') for date in EISDict.keys()]
    x_day = [x_day[i] for i in eis_seq]

    x_train = np.array([(poi - x_day[0]).days for poi in x_day])
    x_eval = np.linspace(0,max(x_train),max(x_train)*SPEED_RATE+1)

    if LOG_FLAG:
        chData_log = np.log(chData[eis_seq,1,:] + 1j*chData[eis_seq,2,:])
        y_train = np.stack([chData_log.real.T,chData_log.imag.T], axis=2)
    else: 
        y_train = np.stack([chData[eis_seq,1,:].T,chData[eis_seq,2,:].T], axis=2)

    y_train = y_train.take(run_list, axis=0)

    Scaler_X        = StandardScaler()
    Scaler_Y_real   = StandardScaler()
    Scaler_Y_imag   = StandardScaler()

    if NORM_Y_FLAG:
        y_train[:,:,0] = Scaler_Y_real.fit_transform(y_train[:,:,0].T).T
        y_train[:,:,1] = Scaler_Y_imag.fit_transform(y_train[:,:,1].T).T
    if NORM_X_FLAG:
        x_train = Scaler_X.fit_transform(x_train.reshape(-1, 1)).flatten()
        x_eval = Scaler_X.transform(x_eval.reshape(-1, 1)).flatten()

    logger.info(f"\nx: {np.shape(x_train)} \ny: {np.shape(y_train)} \nx_pred{np.shape(x_eval)}")

    return x_train, y_train, x_eval, [Scaler_X, Scaler_Y_real, Scaler_Y_imag]


def GPDataExporter(x_train, y_train, x_eval, y_eval_mean, y_eval_var, ScalerSet, NORM_X_FLAG, NORM_Y_FLAG):
    # Export Data
    if NORM_X_FLAG:
        x_train = ScalerSet[0].inverse_transform(x_train.reshape(-1, 1)).flatten()
        x_eval = ScalerSet[0].inverse_transform(x_eval.reshape(-1, 1)).flatten()
    
    if NORM_Y_FLAG:
        y_train_real = ScalerSet[1].inverse_transform(y_train[:,:,0].T).T
        y_train_imag = ScalerSet[2].inverse_transform(y_train[:,:,1].T).T
        
        y_eval_mean_real = ScalerSet[1].inverse_transform(y_eval_mean[:,:,0].T).T
        y_eval_mean_imag = ScalerSet[2].inverse_transform(y_eval_mean[:,:,1].T).T

        y_eval_var_real = y_eval_var[:,:,0] * ScalerSet[1].var_.reshape(-1,1)
        y_eval_var_imag = y_eval_var[:,:,1] * ScalerSet[2].var_.reshape(-1,1)
    else:
        y_train_real = y_train[:,:,0]
        y_train_imag = y_train[:,:,1]

        y_eval_mean_real = y_eval_mean[:,:,0]
        y_eval_mean_imag = y_eval_mean[:,:,1]

        y_eval_var_real = y_eval_var[:,:,0]
        y_eval_var_imag = y_eval_var[:,:,1]

    y_train = np.stack([y_train_real, y_train_imag], axis=2)
    y_eval = np.stack([y_eval_mean_real, y_eval_mean_imag], axis=2)
    y_eval_err = np.stack([y_eval_var_real, y_eval_var_imag], axis=2)
    
    logger.info(f"\nx: {np.shape(x_train)} \ny: {np.shape(y_train)} \nx_pred{np.shape(x_eval)} \ny_pred{np.shape(y_eval)} \ny_pred{np.shape(y_eval_err)}")

    return x_train, y_train, x_eval, y_eval, y_eval_err


### Main Run

In [541]:

SPEED_RATE=1
LOG_FLAG=False
NORM_X_FLAG=True
NORM_Y_FLAG=True

run_list = range(70,80)
# run_list = range(np.shape(chData)[2])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x_train, y_train, x_eval, ScalerSet = \
    GPDataLoader(chData, eis_seq, eis_cluster, run_list,
        SPEED_RATE=1, LOG_FLAG=LOG_FLAG, NORM_X_FLAG=NORM_Y_FLAG, NORM_Y_FLAG=NORM_Y_FLAG)

x_train_tensor = torch.from_numpy(x_train).float().to(device)
x_eval_tensor = torch.from_numpy(x_eval).float().to(device)
y_train_tensor = torch.from_numpy(y_train).float().to(device)



2025-04-04 18:24:54.697 | INFO     | __main__:GPDataLoader:28 - 
x: (11,) 
y: (10, 11, 2) 
x_pred(19,)


In [543]:
y_eval_mean = []
y_eval_var  = []
# for i in range(np.shape(freq_list)[0]):
for i in range(np.shape(y_train)[0]):
    y_eval_tensor, loss_inst, length_inst, noise_inst = \
        ComplexGPTrain(x_train_tensor, y_train_tensor[i,:,:], x_eval_tensor, device, training_iter=200, lr=0.05)
    y_eval_mean.append(y_eval_tensor.mean.cpu().numpy())
    y_eval_var.append(y_eval_tensor.variance.detach().cpu().numpy())

y_eval_mean = np.array(y_eval_mean)
y_eval_var = np.array(y_eval_var)



logger.info(f"\nmean: {np.shape(y_eval_mean)}\nvar: {np.shape(y_eval_var)}")


2025-04-04 18:25:44.071 | INFO     | __main__:ComplexGPTrain:61 - Iter 1/200	Loss: 1.5818606615066528	Length: 0.7184596657752991	Noise: 0.6685596704483032
2025-04-04 18:25:44.229 | INFO     | __main__:ComplexGPTrain:61 - Iter 11/200	Loss: 1.3522344827651978	Length: 1.0023382902145386	Noise: 0.45255815982818604
2025-04-04 18:25:44.336 | INFO     | __main__:ComplexGPTrain:61 - Iter 21/200	Loss: 1.2447978258132935	Length: 1.2496873140335083	Noise: 0.29415324330329895
2025-04-04 18:25:44.438 | INFO     | __main__:ComplexGPTrain:61 - Iter 31/200	Loss: 1.1350525617599487	Length: 1.4189584255218506	Noise: 0.18312792479991913
2025-04-04 18:25:44.530 | INFO     | __main__:ComplexGPTrain:61 - Iter 41/200	Loss: 1.0219449996948242	Length: 1.6528459787368774	Noise: 0.10980099439620972
2025-04-04 18:25:44.616 | INFO     | __main__:ComplexGPTrain:61 - Iter 51/200	Loss: 0.9096075892448425	Length: 1.9307713508605957	Noise: 0.06528928130865097
2025-04-04 18:25:44.699 | INFO     | __main__:ComplexGPTrain

In [ ]:
x_train, y_train, x_eval, y_eval, y_eval_err = \
        GPDataExporter(x_train, y_train, x_eval, y_eval_mean, y_eval_var, ScalerSet,
                       NORM_X_FLAG=NORM_X_FLAG, NORM_Y_FLAG=NORM_Y_FLAG)
    

2025-04-04 18:22:54.558 | INFO     | __main__:GPDataExporter:62 - 
x: (11,) 
y: (10, 11, 2) 
x_pred(19,) 
y_pred(10, 19, 2) 
y_pred(10, 19, 2)


### Loss Plot

In [ ]:
if True:
    fig = plt.figure()
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(313)
    ax1.plot(loss_inst)
    ax1.set_title("Loss")
    ax2.plot(noise_inst)
    ax2.set_yscale('log')
    ax2.set_title("Noise")
    ax3.plot(length_inst)
    ax3.set_yscale('linear')
    ax3.set_title("LengthScale")
    fig.set_tight_layout(True)

### GPR Result Plot

In [ ]:
if True:
    n_freq = np.shape(y_eval)[0]
    fig = plt.figure(figsize=(8,8))
    cmap = plt.colormaps.get_cmap('viridis_r')
    axis0 = fig.add_subplot(2,2,1)
    axis1 = fig.add_subplot(2,2,2)
    axis2 = fig.add_subplot(2,2,3)
    axis3 = fig.add_subplot(2,2,4)


    # for i in range(n_freq):
    # for i in range(70,80):
    for i in range(np.shape(y_eval)[0]):
        axis0.fill_between(x_eval, y_eval[i,:,0] - 2*np.sqrt(y_eval_err[i,:,0]), y_eval[i,:,0] + 2*np.sqrt(y_eval_err[i,:,0]), 
                        alpha=0.2, color = cmap(run_list[i]/n_freq))
        
        axis1.plot(x_eval, y_eval[i,:,0], color = cmap(i/n_freq))
        axis1.plot(x_train, y_train[i,:,0], color = cmap(i/n_freq), linestyle = ' ', marker = 'o')

        axis2.fill_between(x_eval, y_eval[i,:,1] - 2*np.sqrt(y_eval_err[i,:,1]), y_eval[i,:,1] + 2*np.sqrt(y_eval_err[i,:,1]), 
                        alpha=0.2, color = cmap(run_list[i]/n_freq))
        
        axis3.plot(x_eval, y_eval[i,:,1], color = cmap(i/n_freq))
        axis3.plot(x_train, y_train[i,:,1], color = cmap(i/n_freq), linestyle = ' ', marker = 'o')

    # axis1.sharex(axis0)
    # axis1.sharey(axis0)
    # axis3.sharex(axis2)
    # axis3.sharey(axis2)
    axis0.set_xlabel('x')
    axis0.set_ylabel('y')



### GPR EIS Plot

In [ ]:
if True:
    
    n_freq = np.shape(y_eval_mean)[0]
    fig = plt.figure(figsize=(8,8))
    cmap = plt.colormaps.get_cmap('rainbow_r')
    axis0 = fig.add_subplot(2,3,1)
    axis1 = fig.add_subplot(2,3,2)
    axis2 = fig.add_subplot(2,3,3, projection = '3d')
    axis3 = fig.add_subplot(2,3,4)
    axis4 = fig.add_subplot(2,3,5)
    axis5 = fig.add_subplot(2,3,6, projection = '3d')


    init_elev = 30  # 仰角
    init_azim = -40  # 方位角
    axis2.view_init(elev=init_elev, azim=init_azim)
    axis5.view_init(elev=init_elev, azim=init_azim)


    new_f = chData[0,0,:].take(run_list, axis=0)

    if LOG_FLAG:
        
        y_EIS_train =   np.exp(y_train[:,:,0] + 1j * y_train[:,:,1])
        y_EIS_eval = np.exp(y_eval[:,:,0] + 1j * y_eval[:,:,1])

        # f_plot = range(40,80)
        f_plot = range(np.shape(new_f)[0])
        y_EIS_train = y_EIS_train.take(f_plot, axis=0)
        y_EIS_eval = y_EIS_eval.take(f_plot, axis=0)
        new_f = new_f.take(f_plot, axis=0)

    else:
        y_EIS_train = y_train[:,:,0] + 1j*y_train[:,:,1]
        y_EIS_eval = y_eval[:,:,0] + 1j*y_eval[:,:,1]

        f_plot = range(40,60)
        f_plot = range(np.shape(new_f)[0])
        y_EIS_train = y_EIS_train.take(f_plot, axis=0)
        y_EIS_eval = y_EIS_eval.take(f_plot, axis=0)
        new_f = new_f.take(f_plot, axis=0)



    for i in range(np.shape(x_eval)[0]):
        axis1.semilogx(new_f, np.log10(np.abs(y_EIS_eval[:,i])), color = cmap(i/np.shape(x_eval)[0]))
        axis4.semilogx(new_f, np.rad2deg(np.angle(y_EIS_eval[:,i])), color = cmap(i/np.shape(x_eval)[0]))
        
        
    for i in range(np.shape(x_train)[0]):
        axis1.semilogx(new_f, np.log10(np.abs(y_EIS_train[:,i])), 'black', alpha = 0.3)
        axis4.semilogx(new_f, np.rad2deg(np.angle(y_EIS_train[:,i])), 'black', alpha = 0.3)
    


    _y = np.arange(np.shape(x_eval)[0])
    _x = np.log10(new_f).flatten()
    X, Y = np.meshgrid(_x, _y, indexing='ij')
    axis2.plot_surface(X, Y, np.log10(np.abs(y_EIS_eval[:,:])), cmap='viridis_r', alpha=0.8)
    axis5.plot_surface(X, Y, np.rad2deg(np.angle(y_EIS_eval[:,:])) * 180 / np.pi, cmap='viridis_r', alpha=0.8)


